<a href="https://colab.research.google.com/github/tlchampion/simpsons_character_recognition/blob/main/notebooks%5Cmodel_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose

Model 3 will use the hyperband method to perform hyperparameter tuning on the CNN model.

<br>


# Setup Environment

## Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install and import modules

In [2]:
! pip install python-dotenv
%pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 14.8 MB/s 


In [3]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import PIL 
import PIL.Image
import pathlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
sys.path.insert(0,"/content/drive/My Drive/Projects/simpsons_character_recognition/simpsons_images")
from dotenv import find_dotenv, load_dotenv
import datetime
import pytz
import tensorflow_addons as tfa
from tensorflow_addons.metrics import F1Score

In [4]:
from d01_data import make_dataset

In [15]:
pip install keras_tuner

     |████████████████████████████████| 98 kB 6.4 MB/s 


In [18]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras import Sequential
from tensorflow.keras.regularizers import l2
import keras_tuner as kt
from tensorflow import keras

# Copy image files to drive local to colab instance





In [6]:
%cp -r /content/drive/MyDrive/Projects/simpsons_character_recognition/data /content/data

# Import Data

In [7]:
train_directory = os.path.join(os.getcwd(),'data/images/simpsons_dataset')
test_directory = os.path.join(os.getcwd(),'data/images/simpsons_testset')
test_labels = os.path.join(os.getcwd(),'data/images/test_labels.csv')


In [8]:
datagen, datagen_test, train, validation, test = make_dataset.create_datasets(train_directory, test_directory, test_labels)

Found 16764 images belonging to 42 classes.
Found 4169 images belonging to 42 classes.
Found 990 validated image filenames.


# Setup Model

In [9]:
%load_ext tensorboard

In [10]:


early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.001, patience=5, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)

checkpoint_filepath = '/content/drive/MyDrive/Projects/simpsons_character_recognition/checkpoints/model_3/'
checkpoints = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, monitor='val_loss', verbose=0, save_best_only=False,
    save_weights_only=True, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)


log_dir = '/content/drive/MyDrive/Projects/simpsons_character_recognition/logdir/' + 'model_3/' 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


LR_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=5, verbose=0,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0
)

In [23]:
def build_model(hp):
    units1 = hp.Int('units1', min_value=16, max_value=192, step=16)
    units2 = hp.Int('units2', min_value=16, max_value=192, step=16)
    units3 = hp.Int('units3', min_value=16, max_value=192, step=16)
    lr = hp.Choice('learning rate', values=[1e-2, 1e-3, 1e-4])

    model = keras.Sequential()
    model.add(Conv2D(units1, 3, activation='relu'))
    model.add(MaxPool2D(pool_size=2))
    model.add(Conv2D(units2, 3, activation='relu'))
    model.add(MaxPool2D(pool_size=2))
    model.add(Conv2D(units3, 3, activation='relu'))
    model.add(MaxPool2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(42, activation='softmax'))



    f1_metric = tfa.metrics.F1Score(num_classes=42, average='weighted')




    # Compile the model
    model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate=lr),
                metrics=['accuracy', 'AUC', f1_metric])

    return model

In [24]:
tuner = kt.Hyperband(build_model,
                      objective='val_accuracy',
                      max_epochs=200,
                      factor=3,
                      directory="/content/drive/MyDrive/Projects/simpsons_character_recognition/hp",
                      project_name='model3')

In [ ]:
tuner.search(train, 
              validation_data=validation,
              epochs=200,
              callbacks=[tensorboard_callback,early_stop] )


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
units1            |80                |?                 
units2            |64                |?                 
units3            |176               |?                 
learning rate     |0.001             |?                 
tuner/epochs      |3                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |4                 |?                 
tuner/round       |0                 |?                 

Epoch 1/3
524/524 [==============================] - 83s 152ms/step - loss: 2.4846 - accuracy: 0.3218 - auc: 0.8863 - f1_score: 0.2998 - val_loss: 2.3384 - val_accuracy: 0.3838 - val_auc: 0.8955 - val_f1_score: 0.3788
Epoch 2/3
524/524 [==============================] - 79s 152ms/step - loss: 1.5995 - accuracy: 0.5741 - auc: 0.9488 - f1_score: 0.5646 - val_loss: 1.9165 - val_accuracy: 0.5179 - val_auc: 0.9233 - val_f1_score: 0.5079
Epoch 3/3
 24/524 [>...

In [ ]:
%tensorboard --logdir '/content/drive/MyDrive/Projects/simpsons_character_recognition/logdir/'


In [ ]:
tuner.results_summary()

In [ ]:
tuner.get_best_hyperparameters()[0]